In [ ]:
"""
A drop-in image simulation script for abTEM.  Reads in a file in XYZ format and outputs a simulated
image.  Parameters in an editable dictionary at the top of the file, after the file IO.
"""
# IO
from pathlib import Path
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
# AbTEM imports for the main sim loop
from abtem.potentials import Potential
from abtem.structures import orthogonalize_cell
from abtem.temperature import FrozenPhonons
from abtem.waves import Probe
from abtem.scan import GridScan
from abtem.detect import FlexibleAnnularDetector
# Other
from copy import deepcopy
from graphical import gui_open, gui_savetiff
from ase.visualize import view
import matplotlib.pyplot as plt  # Show outputs
from ase import build
from tifffile import imwrite

# %% IO
atoms = gui_open("vasp")
atoms.rotate(90, "y", rotate_cell=True)
rot_atoms = deepcopy(atoms)
ortho_atoms = orthogonalize_cell(rot_atoms)
ortho_atoms *= (1, 1, 1)

# %%

new_atoms = deepcopy(ortho_atoms)
symbols = new_atoms.get_chemical_symbols()
for i, s in enumerate(symbols):
    if s == "Sc":
        symbols[i] = "Y"
new_atoms.set_chemical_symbols(symbols)

# %% PARAMETERS
prms = {"seed":            42,              # Pseudo-random seed
        "device":          "cpu",           # Set computing device, must be "gpu" or "cpu"
        "gpu_num":         1,               # If "device" == "gpu", which GPU to use (0 or 1)
        # STRUCTURE FILE LOCATION
        "path":            r"E:\Users\Charles\BTO PACBED\abtem",
        "filename":        "BaTiO3_mp-2998_conventional_standard.cif",
        # POTENTIAL SETTINGS
        "sampling":        0.05,            # Sampling of potential (A)
        "tiling":          30,              # Number of times to tile projected cell
        "thickness":       200,             # Total model thickness (A)
        "thickness_step":  10,              # PACBED export thickness steps (A)
        "slice_thickness": 2,               # Thickness per simulation slice (A)
        "za":              (1, 1, 0),       # Zone axes to model
        "fp_configs":      10,              # Number of frozen phonon configurations
        "fp_sigmas":       {"Al": 0.0578,   # Frozen phonon sigma values per atom type (A)
                            "Sc": 0.0578,   # sigma == sqrt(U_iso) (confirmed with abTEM author)
                            "N":  0.060},   # Data: 10.1107/S0108767309004966, Sc is wrong
        # PROBE SETTINGS
        "beam_energy":     200E3,           # Energy of the electron beam (eV)
        "convergence":     17.9,            # Probe semiangle of convergence (mrad)
        # DETECTOR SETTINGS
        "max_batch":       200,             # Number of probe positions to propogate at once
        "max_angle":       200}              # Maximum detector angle (mrad)

# %% SETUP
# fps = FrozenPhonons(ortho_atoms,
#                     sigmas=prms["fp_sigmas"],
#                     num_configs=prms["fp_configs"],
#                     seed=prms["seed"])

potential = Potential(new_atoms,
                      sampling=prms["sampling"],
                      device="cpu",
                      storage="cpu",
                      projection="infinite",
                      parametrization="kirkland",
                      precalculate=False)

probe = Probe(energy=prms["beam_energy"],
              semiangle_cutoff=prms["convergence"],
              device="cpu")

scan = GridScan(start=[potential.extent[0]/4, potential.extent[1]/4],
                end=[potential.extent[0]/2, potential.extent[1]/2],
                sampling=probe.ctf.nyquist_sampling)
detector = FlexibleAnnularDetector()
probe.grid.match(potential)
# %% RUN
measurement = probe.scan(scan, detector, potential, pbar=False)

# %% DETECT
meas = deepcopy(measurement)  # Don't mutate original measurements, or must redo sim

haadf = meas.integrate(70, 200)  # Integration limits from camera length; max limited by sampling
# Interpolate using 4xFFT
# Demonstrated by abTEM author to be nearly identical to 4x oversampling, much faster
haadf = haadf.interpolate(tuple([x/4 for x in scan.sampling]), kind="fft")
# %% SHOW
plt.imshow(haadf._array)
plt.title("AlScN")

# %% ANALYZE MODEL

aatoms = deepcopy(new_atoms)
x, y = aatoms.cell[0][0], aatoms.cell[1][1]
xmin, xmax, ymin, ymax = x/4, x/2, y/4, y/2
to_del = []
for i, atom in enumerate(aatoms):
    if not (atom.position[0] > xmin and atom.position[0] < xmax and
            atom.position[1] > ymin and atom.position[1] < ymax):
        to_del.append(i)
del aatoms[to_del]

# %% SAVE
gui_savetiff(haadf)
